In [ ]:
pip install mysql-connector-python

In [ ]:
import mysql.connector
from mysql.connector import Error
import json

# MySQL 연결 설정 함수
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

# 데이터 가져오기 함수
def fetch_data(connection, query):
    cursor = connection.cursor(dictionary=True)
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

# MySQL 연결
connection = create_connection("RDS주소", "username", "password", "table")

# SQL 쿼리 수정: AnilistAnime 테이블에서 한글 제목과 설명도 가져옴
query = """
SELECT a.anime_id, a.anime_title, a.anime_title_ko, c.genre_name, e.tag_name, f.popularity
FROM Anime a
JOIN AniGenre b ON a.anime_id = b.anime_id
JOIN Genre c ON b.genre_id = c.genre_id
JOIN AniTag d ON a.anime_id = d.anime_id
JOIN Tag e ON d.tag_id = e.tag_id
JOIN AnilistAnime f ON a.anime_id = f.anime_id;
"""
datas = fetch_data(connection, query)

# 결과를 저장할 딕셔너리 초기화
result_dict = {}

# 데이터를 딕셔너리에 저장
for row in datas:
    anime_id = row['anime_id']
    anime_title = row['anime_title']
    anime_title_ko = row['anime_title_ko']
    genre_name = row['genre_name']
    tag_name = row['tag_name']
    popularity = row['popularity']

    if anime_id not in result_dict:
        result_dict[anime_id] = {
            'id': anime_id,
            'anime_title': anime_title,
            'anime_title_ko': anime_title_ko,
            'genre': [genre_name],
            'tag': [tag_name],
            'popularity': popularity
        }
    else:
        if genre_name not in result_dict[anime_id]['genre']:
            result_dict[anime_id]['genre'].append(genre_name)
        if tag_name not in result_dict[anime_id]['tag']:
            result_dict[anime_id]['tag'].append(tag_name)

# 딕셔너리를 리스트로 변환
result_list = list(result_dict.values())

# 결과를 JSON 파일로 저장
file_path = "anime_data.json"
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(result_list, json_file, ensure_ascii=False, indent=4)

print(f"데이터가 {file_path}에 저장되었습니다.")

Connection to MySQL DB successful
데이터가 anime_data.json에 저장되었습니다.
